In [18]:
from urllib.parse import urljoin

import pandas as pd
import requests
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

import os
import subprocess
import zipfile
import json
from collections import defaultdict, Counter
import unicodedata
from googletrans import Translator
import tqdm
import time
from urllib.parse import urlsplit, urlunsplit
import imghdr
import urllib
import html

In [19]:
def clean_string(s):
    return html.unescape(
        unicodedata.normalize('NFKD', s)
            .replace('(', ' (')
            .replace(')', ') ')
            .replace('（', ' (')
            .replace('）', ') ')
            .replace('—', ' ')
            .replace('°', ' ')
            .replace('  ', ' ')
            .replace('   ', ' ')
            .strip()
            # Replace for safe csv saving
            .replace(';', ',')
    )

def is_english(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
    
def remove_query_params_and_fragment(url):
    return urlunsplit(urlsplit(url)._replace(query="", fragment=""))

def reverse_dict(mydict):
    reversed_dict = defaultdict(list)
    for key, value in mydict.items():
        reversed_dict[value].append(key)
        
    return reversed_dict
    
translator = Translator(service_urls=[
  'translate.google.com',
])

In [13]:
FILE_PATH_RAW_CATEGORIES

'../data/raw/recommend_deivce_name.json'

In [14]:
VEVS_BASE_URL = 'https://nl.kapiba.ru/mihome/files/old/'
VEVS_PASSWORD = '344022856'

ASSETS_PATH = '../assets'
ASSETS_ICONS_RAW_PATH = os.path.join(ASSETS_PATH, 'icons', 'raw')
ASSETS_ICONS_PROCESSED_PATH = os.path.join(ASSETS_PATH, 'icons', 'processed')

DATA_PATH_ROOT = '../data'
DATA_PATH_RAW = os.path.join(DATA_PATH_ROOT, 'raw')
DATA_PATH_PROCESSED = os.path.join(DATA_PATH_ROOT, 'processed')
DATA_PATH_MANUAL = os.path.join(DATA_PATH_ROOT, 'manual')

FILE_NAME_RAW_DEVICES = 'plugin_config_all'
FILE_PATH_RAW_DEVICES = os.path.join(DATA_PATH_RAW, FILE_NAME_RAW_DEVICES)
FILENAME_RAW_CATEGORIES = 'recommend_deivce_name.json'
FILE_PATH_RAW_CATEGORIES = os.path.join(DATA_PATH_RAW, FILENAME_RAW_CATEGORIES)

FILE_NAME_PROCESSED_DEVICES = 'devices.csv'
FILE_PATH_PROCESSED_DEVICES = os.path.join(DATA_PATH_PROCESSED, FILE_NAME_PROCESSED_DEVICES)
FILE_NAME_PROCESSED_CATEGORIES = 'categories.csv'
FILE_PATH_PROCESSED_CATEGORIES = os.path.join(DATA_PATH_PROCESSED, FILE_NAME_PROCESSED_CATEGORIES)

FILE_NAME_PROCESSED_SUBCATE_CATE_DICT = 'subcate_cate.json'
FILE_PATH_PROCESSED_SUBCATE_CATE_DICT = os.path.join(DATA_PATH_PROCESSED, FILE_NAME_PROCESSED_SUBCATE_CATE_DICT)

FILE_NAME_PROCESSED_NAME_TRANSLATIONS = 'name_translations.json'
FILE_PATH_PROCESSED_NAME_TRANSLATIONS = os.path.join(DATA_PATH_PROCESSED, FILE_NAME_PROCESSED_NAME_TRANSLATIONS)
FILE_NAME_PROCESSED_DESC_TRANSLATIONS = 'desc_translations.json'
FILE_PATH_PROCESSED_DESC_TRANSLATIONS = os.path.join(DATA_PATH_PROCESSED, FILE_NAME_PROCESSED_DESC_TRANSLATIONS)

FILE_NAME_MANUAL_PRODUCT_NUMBERS_RAW = 'product_number_raw.csv'
FILE_PATH_MANUAL_PRODUCT_NUMBERS_RAW = os.path.join(DATA_PATH_MANUAL, FILE_NAME_MANUAL_PRODUCT_NUMBERS_RAW)

FILE_NAME_PROCESSED_PRODUCT_NUMBERS = 'product_number.csv'
FILE_PATH_PROCESSED_PRODUCT_NUMBERS = os.path.join(DATA_PATH_PROCESSED, FILE_NAME_PROCESSED_PRODUCT_NUMBERS)

ARCHIVE_PATH_DEVICES = 'assets/vevs/config_all.json'
ARCHIVE_PATH_CATEGORIES = 'assets/cardControl/recommend_deivce_name.json'

REGEXP_STR_PRODUCT_CODE = "[A-Z]{2,}[a-z]{0,2}[0-9\-\/]{2,}[a-z]{0,2}[a-zA-Z0-9\-\/]+"
REGEXP_STR_MODEL_ID = "^[a-z0-9_]+\.[a-z0-9_]+\.[a-z0-9_]+$"

# Extract Data

##### Download MiHome APK

In [6]:
!brew install wget

==> Downloading https://ghcr.io/v2/homebrew/core/libunistring/manifests/1.0
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/libunistring/blobs/sha256:b8b2f
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sh
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/libidn2/manifests/2.3.2
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/libidn2/blobs/sha256:fcc51d2c38
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sh
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/wget/manifests/1.21.2
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/wget/bl

In [11]:
s = requests.Session()
s.auth = ('', VEVS_PASSWORD)
page = s.get(VEVS_BASE_URL)

# # Select latest file
# df = pd.read_html(page.content)[0]
# vevs_package_filename = (
#     df.loc[df['Type'] == 'application/vnd.android.package-archive']
#     .tail(1)['Name']
#     .tolist()[0]
# )

vevs_package_filename = "MiHome_6.14.705_65110_vevs.apk"

s = f'wget --user "" --password {VEVS_PASSWORD} --read-timeout=5 --tries=0 --no-clobber -P {DATA_PATH_RAW} {urljoin(VEVS_BASE_URL, vevs_package_filename)}'
output = subprocess.check_output(s, shell=True)

--2022-01-12 23:27:21--  https://nl.kapiba.ru/mihome/files/old/MiHome_6.14.705_65110_vevs.apk
Resolving nl.kapiba.ru (nl.kapiba.ru)... 94.103.80.24
Connecting to nl.kapiba.ru (nl.kapiba.ru)|94.103.80.24|:443... connected.
HTTP request sent, awaiting response... 401 Unauthorized
Authentication selected: Basic realm="PLEASE ENTER THE PASSWORD, NO USERNAME NEEDED", charset="UTF-8"
Reusing existing connection to nl.kapiba.ru:443.
HTTP request sent, awaiting response... 200 OK
Length: 139671679 (133M) [application/vnd.android.package-archive]
Saving to: ‘../data/raw/MiHome_6.14.705_65110_vevs.apk’

     0K .......... .......... .......... .......... ..........  0%  414K 5m29s
    50K .......... .......... .......... .......... ..........  0%  833K 4m6s
   100K .......... .......... .......... .......... ..........  0% 10,3M 2m48s
   150K .......... .......... .......... .......... ..........  0% 9,94M 2m10s
   200K .......... .......... .......... .......... ..........  0%  890K 2m14s
   25

  5650K .......... .......... .......... .......... ..........  4% 3,18M 24s
  5700K .......... .......... .......... .......... ..........  4%  116M 24s
  5750K .......... .......... .......... .......... ..........  4% 2,40M 24s
  5800K .......... .......... .......... .......... ..........  4% 13,8M 24s
  5850K .......... .......... .......... .......... ..........  4% 9,99M 24s
  5900K .......... .......... .......... .......... ..........  4% 6,18M 24s
  5950K .......... .......... .......... .......... ..........  4% 8,03M 24s
  6000K .......... .......... .......... .......... ..........  4% 9,33M 24s
  6050K .......... .......... .......... .......... ..........  4% 3,34M 24s
  6100K .......... .......... .......... .......... ..........  4% 8,84M 24s
  6150K .......... .......... .......... .......... ..........  4% 4,28M 24s
  6200K .......... .......... .......... .......... ..........  4% 7,76M 24s
  6250K .......... .......... .......... .......... ..........  4% 9,72M 23s

 11000K .......... .......... .......... .......... ..........  8% 2,60M 24s
 11050K .......... .......... .......... .......... ..........  8% 4,12M 24s
 11100K .......... .......... .......... .......... ..........  8% 2,03M 24s
 11150K .......... .......... .......... .......... ..........  8%  303M 24s
 11200K .......... .......... .......... .......... ..........  8% 2,32M 24s
 11250K .......... .......... .......... .......... ..........  8% 2,91M 24s
 11300K .......... .......... .......... .......... ..........  8% 2,05M 25s
 11350K .......... .......... .......... .......... ..........  8% 8,40M 25s
 11400K .......... .......... .......... .......... ..........  8% 5,88M 24s
 11450K .......... .......... .......... .......... ..........  8% 2,80M 25s
 11500K .......... .......... .......... .......... ..........  8% 5,55M 25s
 11550K .......... .......... .......... .......... ..........  8% 2,00M 25s
 11600K .......... .......... .......... .......... ..........  8% 7,12M 25s

 16400K .......... .......... .......... .......... .......... 12% 2,28M 26s
 16450K .......... .......... .......... .......... .......... 12% 6,67M 26s
 16500K .......... .......... .......... .......... .......... 12% 3,04M 26s
 16550K .......... .......... .......... .......... .......... 12% 5,63M 26s
 16600K .......... .......... .......... .......... .......... 12% 4,67M 26s
 16650K .......... .......... .......... .......... .......... 12% 2,43M 26s
 16700K .......... .......... .......... .......... .......... 12% 11,1M 26s
 16750K .......... .......... .......... .......... .......... 12% 3,54M 26s
 16800K .......... .......... .......... .......... .......... 12% 3,20M 26s
 16850K .......... .......... .......... .......... .......... 12% 5,00M 26s
 16900K .......... .......... .......... .......... .......... 12% 2,44M 26s
 16950K .......... .......... .......... .......... .......... 12% 13,1M 26s
 17000K .......... .......... .......... .......... .......... 12% 3,07M 26s

 22000K .......... .......... .......... .......... .......... 16% 3,28M 27s
 22050K .......... .......... .......... .......... .......... 16% 3,73M 27s
 22100K .......... .......... .......... .......... .......... 16% 1,99M 27s
 22150K .......... .......... .......... .......... .......... 16% 5,24M 27s
 22200K .......... .......... .......... .......... .......... 16% 4,00M 27s
 22250K .......... .......... .......... .......... .......... 16% 4,04M 27s
 22300K .......... .......... .......... .......... .......... 16% 3,46M 27s
 22350K .......... .......... .......... .......... .......... 16% 2,20M 27s
 22400K .......... .......... .......... .......... .......... 16% 4,07M 27s
 22450K .......... .......... .......... .......... .......... 16% 3,69M 27s
 22500K .......... .......... .......... .......... .......... 16% 2,17M 27s
 22550K .......... .......... .......... .......... .......... 16% 3,67M 27s
 22600K .......... .......... .......... .......... .......... 16% 6,13M 27s

...

111600K .......... .......... .......... .......... .......... 81% 4,25M 8s
111650K .......... .......... .......... .......... .......... 81% 1,15M 8s
111700K .......... .......... .......... .......... .......... 81% 6,55M 8s
111750K .......... .......... .......... .......... .......... 81% 56,2M 8s
111800K .......... .......... .......... .......... .......... 82% 7,49M 8s
111850K .......... .......... .......... .......... .......... 82% 6,24M 8s
111900K .......... .......... .......... .......... .......... 82% 1,23M 8s
111950K .......... .......... .......... .......... .......... 82% 5,78M 8s
112000K .......... .......... .......... .......... .......... 82% 6,31M 8s
112050K .......... .......... .......... .......... .......... 82% 30,0M 8s
112100K .......... .......... .......... .......... .......... 82% 1,11M 8s
112150K .......... .......... .......... .......... .......... 82% 7,53M 8s
112200K .......... .......... .......... .......... .......... 82% 7,29M 8s
112250K ....

117550K .......... .......... .......... .......... .......... 86% 2,50M 6s
117600K .......... .......... .......... .......... .......... 86% 2,87M 6s
117650K .......... .......... .......... .......... .......... 86% 7,67M 6s
117700K .......... .......... .......... .......... .......... 86% 1,33M 6s
117750K .......... .......... .......... .......... .......... 86% 8,10M 6s
117800K .......... .......... .......... .......... .......... 86% 3,53M 6s
117850K .......... .......... .......... .......... .......... 86% 5,09M 6s
117900K .......... .......... .......... .......... .......... 86% 1,53M 6s
117950K .......... .......... .......... .......... .......... 86% 3,67M 6s
118000K .......... .......... .......... .......... .......... 86% 5,59M 6s
118050K .......... .......... .......... .......... .......... 86% 2,55M 6s
118100K .......... .......... .......... .......... .......... 86% 2,10M 6s
118150K .......... .......... .......... .......... .......... 86% 3,29M 6s
118200K ....

123350K .......... .......... .......... .......... .......... 90% 12,5M 4s
123400K .......... .......... .......... .......... .......... 90% 2,54M 4s
123450K .......... .......... .......... .......... .......... 90% 2,08M 4s
123500K .......... .......... .......... .......... .......... 90% 2,62M 4s
123550K .......... .......... .......... .......... .......... 90% 13,6M 4s
123600K .......... .......... .......... .......... .......... 90% 2,95M 4s
123650K .......... .......... .......... .......... .......... 90% 1,78M 4s
123700K .......... .......... .......... .......... .......... 90% 1,95M 4s
123750K .......... .......... .......... .......... .......... 90%  110M 4s
123800K .......... .......... .......... .......... .......... 90% 6,84M 4s
123850K .......... .......... .......... .......... .......... 90% 2,19M 4s
123900K .......... .......... .......... .......... .......... 90% 1,55M 4s
123950K .......... .......... .......... .......... .......... 90% 7,18M 4s
124000K ....

128800K .......... .......... .......... .......... .......... 94% 1,99M 2s
128850K .......... .......... .......... .......... .......... 94% 2,40M 2s
128900K .......... .......... .......... .......... .......... 94% 4,70M 2s
128950K .......... .......... .......... .......... .......... 94% 6,84M 2s
129000K .......... .......... .......... .......... .......... 94% 4,43M 2s
129050K .......... .......... .......... .......... .......... 94% 3,31M 2s
129100K .......... .......... .......... .......... .......... 94% 2,28M 2s
129150K .......... .......... .......... .......... .......... 94% 5,49M 2s
129200K .......... .......... .......... .......... .......... 94% 1,99M 2s
129250K .......... .......... .......... .......... .......... 94% 11,2M 2s
129300K .......... .......... .......... .......... .......... 94% 1,92M 2s
129350K .......... .......... .......... .......... .......... 94% 5,00M 2s
129400K .......... .......... .......... .......... .......... 94% 2,06M 2s
129450K ....

134200K .......... .......... .......... .......... .......... 98% 2,13M 1s
134250K .......... .......... .......... .......... .......... 98% 5,89M 1s
134300K .......... .......... .......... .......... .......... 98% 3,20M 1s
134350K .......... .......... .......... .......... .......... 98% 1,75M 1s
134400K .......... .......... .......... .......... .......... 98% 2,86M 1s
134450K .......... .......... .......... .......... .......... 98% 4,61M 1s
134500K .......... .......... .......... .......... .......... 98% 4,57M 1s
134550K .......... .......... .......... .......... .......... 98% 1,98M 1s
134600K .......... .......... .......... .......... .......... 98% 2,99M 1s
134650K .......... .......... .......... .......... .......... 98% 4,76M 1s
134700K .......... .......... .......... .......... .......... 98% 3,03M 1s
134750K .......... .......... .......... .......... .......... 98% 2,55M 1s
134800K .......... .......... .......... .......... .......... 98% 2,72M 1s
134850K ....

##### Extract Data from APK

In [15]:
# # You can also read directly from the APK
# archive = zipfile.ZipFile(vevs_package_filename, 'r')
# archive.read(ARCHIVE_PATH_DEVICES)

with zipfile.ZipFile(os.path.join(DATA_PATH_RAW, vevs_package_filename), 'r') as z:
    with open(os.path.join(FILE_PATH_RAW_DEVICES), 'wb') as f:
        f.write(z.read(ARCHIVE_PATH_DEVICES))
        
    with open(os.path.join(FILE_PATH_RAW_CATEGORIES), 'wb') as f:
        f.write(z.read(ARCHIVE_PATH_CATEGORIES))

##### Load Devices data

In [20]:
with open(FILE_PATH_RAW_DEVICES, 'r') as f:
    devices_json = json.load(f)
devices_pd = pd.DataFrame.from_dict(devices_json['result']['devices']['list'])
devices_pd['name'] = devices_pd['name'].apply(clean_string)
devices_pd['desc'] = devices_pd['desc'].apply(clean_string)
devices_pd['model'] = devices_pd['model'].apply(clean_string)
devices_pd = devices_pd.rename(columns={"desc": "desc_cn"})
# devices_pd.head()

# Translate

##### Devices names

In [21]:
with open(FILE_PATH_PROCESSED_NAME_TRANSLATIONS, 'r') as f:
    devices_name_translation_dict = json.load(f)

In [24]:
devices_name_cn_list = []
devices_name_en_list = []

for index, row in tqdm.notebook.tqdm(devices_pd.iterrows(), total=devices_pd.shape[0]):
    if row['name'] and not is_english(row['name']):
        name_cn = row['name']
        
        if name_cn in devices_name_translation_dict:
            name_en = devices_name_translation_dict[name_cn]
        else:
            name_en = translator.translate(name_cn, src='zh-cn', dest='en').text
            devices_name_translation_dict[name_cn] = name_en
            time.sleep(2)
    else:
        name_cn = ''
        name_en = row['name']

    devices_name_cn_list.append(name_cn)
    devices_name_en_list.append(name_en)
    
devices_pd['name_cn'] = devices_name_cn_list
devices_pd['name_en'] = devices_name_en_list

  0%|          | 0/4281 [00:00<?, ?it/s]

In [57]:
# for name_cn, name_en in zip(devices_name_cn_list, devices_name_en_list):
#     if name_cn and name_cn not in devices_name_translation_dict:
#         devices_name_translation_dict[name_cn] = name_en

In [25]:
with open(FILE_PATH_PROCESSED_NAME_TRANSLATIONS, 'w') as f:
    json.dump(devices_name_translation_dict, f)

##### Devices descriptions

In [26]:
with open(FILE_PATH_PROCESSED_DESC_TRANSLATIONS, 'r') as f:
    devices_desc_translation_dict = json.load(f)

In [27]:
devices_desc_en_list = []

for index, row in tqdm.notebook.tqdm(devices_pd.iterrows(), total=devices_pd.shape[0]):
    desc_en = ''
    desc_cn = row['desc_cn']
    
    if desc_cn: 
        if not is_english(desc_cn):
            if desc_cn in devices_desc_translation_dict:
                desc_en = devices_desc_translation_dict[desc_cn]
            else:
                desc_en = translator.translate(desc_cn, src='zh-cn', dest='en').text
                devices_desc_translation_dict[desc_cn] = desc_en
                time.sleep(2)
        else:
            desc_en = row['desc_cn']

    devices_desc_en_list.append(desc_en)
    
devices_pd['desc_en'] = devices_desc_en_list

  0%|          | 0/4281 [00:00<?, ?it/s]

In [61]:
# devices_desc_translation_dict = {}

# for desc_cn, desc_en in zip(list(devices_pd['desc_cn'].values), list(devices_pd['desc_en'].values)):
#     if desc_cn and desc_cn not in devices_desc_translation_dict:
#         devices_desc_translation_dict[desc_cn] = desc_en

In [35]:
with open(FILE_PATH_PROCESSED_DESC_TRANSLATIONS, 'w') as f:
    json.dump(devices_desc_translation_dict, f)

# Create Metacategories

**Original dependencies hell looks like that:**
- subcategory_id (one) -> microcategory_name (many)
- microcategory_name (one) -> devices (many) -> subcategory_id (many)
- cate_name (one) -> subcategory_id (many)

**Our Metasubcategory transformation:**

If device not exists in:
- microcategory_name (one) -> devices (many)

Then:
- microcategory_name (one) -> subcategory_id (one, most popular)
- subcategory_id (one) -> microcategory_name (one, most popular)

In [37]:
with open(FILE_PATH_RAW_CATEGORIES, 'r') as f:
    category_name_json = json.load(f)
category_name_pd = pd.DataFrame.from_dict(category_name_json['data'])
category_name_pd['models'] = category_name_pd['product_type'].apply(lambda x: list(x['model']))
category_name_pd['name_en'] = category_name_pd['product_type'].apply(lambda x: x['defaultName']['en'] if 'en' in x['defaultName'] else translator.translate(x['defaultName'].get('zh_CN'), src='zh-cn', dest='en').text)
category_name_pd['name_en'] = category_name_pd['name_en'].apply(lambda x: clean_string(x).capitalize())
category_name_pd.to_csv(FILE_PATH_PROCESSED_CATEGORIES)
# category_name_pd.head()

In [38]:
model_subcategory_id_dict = pd.Series(devices_pd.subcategory_id.values, index=devices_pd.model).to_dict()
model_cate_name_dict = pd.Series(devices_pd.cate_name.values, index=devices_pd.model).to_dict()

In [39]:
model_microcategory_name_dict = {}
subcategory_id_microcategory_names_dict = defaultdict(list)
subcategory_id_cate_names_dict = defaultdict(list)
# a = defaultdict(list)

for index, row in category_name_pd.iterrows():
    microcategory_name = row['name_en']
    
    subcategories_ids = []
    cate_names = []
    microcategories_names = []
    for m in row['models']:
        if m in model_subcategory_id_dict:
            subcategories_ids.append(model_subcategory_id_dict[m])
            
            if model_cate_name_dict[m] != 'Other':
                cate_names.append(model_cate_name_dict[m])

            model_microcategory_name_dict[m] = microcategory_name
            microcategories_names.append(microcategory_name)
            
    subcategory_id_microcategory_names_dict[Counter(subcategories_ids).most_common()[0][0]].extend(microcategories_names)
#     a[Counter(microcategories_names).most_common()[0][0]].extend(cate_names)
    subcategory_id_cate_names_dict[Counter(subcategories_ids).most_common()[0][0]].extend(cate_names)
    
subcategory_id_microcategory_name_dict = {}
for k,v in subcategory_id_microcategory_names_dict.items():
    subcategory_id_microcategory_name_dict[k] = Counter(v).most_common()[0][0]
    
subcategory_id_cate_name_dict = {}
for k,v in subcategory_id_cate_names_dict.items():
    if len(v):
        subcategory_id_cate_name_dict[k] = Counter(v).most_common()[0][0]
        
# b = {}
# for k,v in a.items():
#     if len(v):
#         b[k] = Counter(v).most_common()[0][0]

In [40]:
def get_metasubcategory_name(s):
    if s['model'] in model_microcategory_name_dict:
        r = model_microcategory_name_dict[s['model']]
    elif s['subcategory_id'] in subcategory_id_microcategory_name_dict:
        r = subcategory_id_microcategory_name_dict[s['subcategory_id']]
    else:
        r = 'Other'
    return r

def get_metacategory_name(s):
    metacategory_name = s['cate_name']
    subcategory_id = s['subcategory_id']
    if (subcategory_id in subcategory_id_cate_name_dict):
#         if subcategory_id_cate_name_dict[subcategory_id] != s['cate_name']:
#             print(s['name_en'], '|||', s['metasubcategory_name'], '|||', subcategory_id_cate_name_dict[subcategory_id], '-', s['cate_name'])
            
        metacategory_name = subcategory_id_cate_name_dict[subcategory_id]
    return metacategory_name

devices_pd['metasubcategory_name'] = devices_pd.apply(get_metasubcategory_name, axis=1)
devices_pd['metacategory_name'] = devices_pd.apply(get_metacategory_name, axis=1)
print(
    devices_pd['metasubcategory_name'].value_counts(True)['Other'],
    devices_pd['cate_name'].value_counts(True)['Other'],
    devices_pd['metacategory_name'].value_counts(True)['Other']
)

0.013548236393366036 0.25718290119131043 0.11539359962625555


In [ ]:
# devices_pd[['name_en', 'cate_name', 'metacategory_name', 'metasubcategory_name', 'subcategory_id']][devices_pd['cate_name'] == 'Other']

In [ ]:
# Wrong matching name ! LEGACY

# # Model - Subcategory ID
# model_subcategory_id_dict = pd.Series(devices_pd.subcategory_id.values, index=devices_pd.model).to_dict()

# # Microcategory Name - Subcategory ID
# microcategory_name_subcategory_ids_dict = defaultdict(list)

# for index, row in category_name_pd.iterrows():
#     for m in row['models']:
#         if m in model_subcategory_id_dict:
#             microcategory_name_subcategory_ids_dict[row['name_en']].append(model_subcategory_id_dict[m])

# microcategory_name_subcategory_id_dict = {}    
# for subcategory_name, subcategory_ids in subcategory_name_subcategory_ids_dict.items():
#     microcategory_name_subcategory_id_dict[subcategory_name] = Counter(subcategory_ids).most_common()[0][0]
    
# # Subcategory ID - Subcategory Name
# subcategory_id_subcategory_name_dict = reverse_dict(subcategory_name_subcategory_id_dict)
    
# # Subcategory Name - Category Name
# devices_pd['subcategory_name'] = devices_pd['subcategory_id'].apply(lambda x: subcategory_id_subcategory_name_dict.get(x, 'Unknown'))
# subcate_cate_dict = (
#     devices_pd[devices_pd['subcategory_name'] != 'Unknown']
#         .groupby('subcategory_name').apply(lambda x: Counter(x['cate_name']).most_common()[0][0]).to_dict()
# )

In [41]:
devices_pd.shape

(4281, 64)

# Clean devices

In [42]:
# Remove all virtual devices
devices_pd = devices_pd[~(
    devices_pd['name_en'].str.contains('virtual', case=False) |
    devices_pd['desc_en'].str.contains('virtual', case=False) |
    devices_pd['model'].str.contains('virtual', case=False) | 
    devices_pd['model'].str.contains('vtl', case=False)
)]

# Remove strange devices that contain foreign model_id in description
devices_pd = devices_pd[
    devices_pd.apply(lambda x: x['desc_en'] not in set(devices_pd['model'].unique()), axis=1)
]

# Swap desc_en -> model_id for others
condition = devices_pd['desc_en'].str.contains(REGEXP_STR_MODEL_ID, case=True, regex=True)
devices_pd.loc[condition, 'model'] = devices_pd[condition]['desc_en']
devices_pd.loc[condition, 'desc_en'] = ''

devices_pd.shape

(4211, 64)

# Product Number

### Automatic (from descriptions)

In [43]:
regexp_exclusion_product_numbers = [
    'ikecin.airfresh.95',
    'rotai.magic_touch.sx300',
    'rotai.massage.rt5870',
    'rotai.massage.rt5870',
    'viomi.dishwasher.v12',
    'viomi.aircondition.v21',
    'isleep.blanket.hs2001',
    'lemesh.switch.sw2a02',
    'viomi.airpurifier.v2',
    'lemesh.switch.sw1a02',
    'tsd.light.test01',
    'tsd.light.tp1',
    'linked.light.ft155q',
    'innolinks.plug.ap3200',
    'ezhome.switch.z4002',
    'wainft.light.wy0a01',
    'viomi.airpurifier.v1',
    'linked.light.ft179',
    'linked.airpurifier.800fac',
    'ezhome.switch.z4003',
    'viomi.bhf_light.v4',
    'yeelink.light.ceiling15',
    'kuju.plug.36',
    'linp.gateway.n2',
    'tsd.light.tsl001',
    'imou99.camera.tp2',
    'jihisi.light.wy0a01',
    'lemesh.switch.sw3a02',
    'linked.light.ft154d',
    'ezhome.switch.z4001',
    'viomi.fan.v5',
    'knx.light.ktexxd',
    'scds.light.wy0a01',
    'xiaomi.aircondition.ma5',
    'ikea.light.led1537r6',
    'lumi.acpartner.v3',
    'scmkcz.switch.cr2p',
    'ikea.light.led1650r5'
]

In [44]:
# REGEXP_STR_PRODUCT_CODE = '[0-9]{0,4}[A-Z]{2,}[\/\\\-]*[a-z\/\-]{0,2}[0-9\/]{1,}[\-]*[a-z]{0,2}[a-zA-Z0-9\-\/]+'
REGEXP_STR_PRODUCT_CODE = "[A-Z]{2,}[a-z]{0,2}[0-9\-\/]{2,}[a-z]{0,2}[a-zA-Z0-9\-\/]+"
devices_pd['product_number'] = devices_pd['desc_en'].str.findall(REGEXP_STR_PRODUCT_CODE).apply(lambda x: ', '.join(set(x)))
devices_pd['product_number'] = devices_pd.apply(lambda x: '' if (x['model'] in regexp_exclusion_product_numbers or 'ESP' in x['product_number']) else x['product_number'], axis=1)
devices_pd[['name_en', 'desc_en', 'model', 'product_number']][devices_pd['product_number'].str.contains(REGEXP_STR_PRODUCT_CODE)]

product_number_auto_dict = dict(
    devices_pd[
        devices_pd['product_number'] != ''
    ][['model', 'product_number']].values
)
len(product_number_auto_dict)

163

### Manual

In [45]:
product_number_manual_raw_pd = pd.read_csv(FILE_PATH_MANUAL_PRODUCT_NUMBERS_RAW, index_col=0)

In [46]:
product_number_manual_raw_pd.loc[product_number_manual_raw_pd['product_number'].isin(['-', '?']), 'product_additional'] = product_number_manual_raw_pd.loc[product_number_manual_raw_pd['product_number'].isin(['-', '?']), 'product_number']
product_number_manual_raw_pd.loc[product_number_manual_raw_pd['product_number'].isin(['-', '?']), 'product_number'] = pd.NA
# product_number_manual_raw_pd.loc[product_number_manual_raw_pd['product_additional'].isin(['-', '?']), 'product_additional']

In [47]:
product_number_manual_pd = product_number_manual_raw_pd[
        (~product_number_manual_raw_pd['product_number'].isnull() | ~product_number_manual_raw_pd['product_additional'].isnull())
][['model_id', 'product_number', 'product_additional']].reset_index(drop=True)
product_number_manual_pd['product_number_source_type'] = 'manual'
# product_number_manual_pd['product_number'] = product_number_manual_pd['product_number'].fillna('').astype('str')
len(product_number_manual_pd)

396

In [48]:
product_number_manual_dict = dict(
    product_number_manual_pd[
        product_number_manual_pd['product_number'].notnull()
    ][['model_id', 'product_number']].values
)
len(product_number_manual_dict)

368

### Hardcode (temporaly manual)

In [49]:
product_number_hardcode_dict = {
    'lumi.switch.b2naus01': 'WS-USC04',
    'lumi.light.cwjwcn01': 'JWSP001A',
    'lumi.switch.b1laus01': 'WS-USC01',
    'lumi.light.cbacn1': 'HLQDQ01LM',
    'lumi.light.rgbac1': 'ZNTGMK11LM',
    'lumi.switch.b2laus01': 'WS-USC02',
    'lumi.switch.l2aeu1':  'WS-EUK02',
    'lumi.light.cwjwcn02' :'JWDL001A',
}

### Union

In [50]:
product_number_dict = {}
product_number_dict.update(product_number_auto_dict)
product_number_dict.update(product_number_manual_dict)
product_number_dict.update(product_number_hardcode_dict)
len(product_number_dict)

443

In [51]:
product_number_pd = pd.DataFrame(product_number_dict.items(), columns=['model_id', 'product_number'])

def product_number_source_type(d):
    t = ''
    if d['model_id'] in product_number_auto_dict:
        t = 'auto'
    elif d['model_id'] in product_number_manual_dict or d['model_id'] in product_number_hardcode_dict:
        t = 'manual'

    return t

product_number_pd['product_number_source_type'] = product_number_pd.apply(
    lambda x: product_number_source_type(x),
    axis=1
)
product_number_pd = product_number_pd.merge(product_number_manual_pd[['model_id', 'product_additional']], how='outer', on='model_id')
product_number_pd['product_number_source_type'].fillna('manual', inplace=True)
product_number_pd = product_number_pd.sort_values('product_number_source_type', ascending=False).reset_index(drop=True)

# assert product_number_pd.shape[0] == len(product_number_pd['model_id'].unique())

In [52]:
product_number_pd

,model_id,product_number,product_number_source_type,product_additional
0,lumi.light.cwjwcn01,JWSP001A,manual,NaN
1,kxf321.mop.mo002,ZDG300,manual,NaN
2,zhimi.aircondition.ma3,KFR-35GW-B1ZM-M3,manual,NaN
3,yeelink.light.mono5,YLDP12YL,manual,NaN
4,lumi.switch.b2laus01,WS-USC02,manual,Double Wall Switch US
5,dmaker.airfresh.t2017,MJXFJ-300-G1,manual,NaN
6,lumi.remote.b486opcn01,WXCJKG12LM,manual,Opple Four Button
7,yeelink.light.bslamp1,MJCTD01YL,manual,NaN
8,lumi.lock.bzacn1,ZNMS17LM,manual,NaN
9,lumi.remote.b286acn02,WXKG07LM,manual,Double Wall Button D1


In [53]:
product_number_pd.to_csv(FILE_PATH_PROCESSED_PRODUCT_NUMBERS)

In [54]:
devices_pd.drop('product_number', inplace=True, axis=1)
devices_pd = devices_pd.merge(
    product_number_pd, 
    how='left', 
    left_on=['model'], 
    right_on=['model_id']
).drop('model_id', axis=1)
# assert devices_pd[~devices_pd['product_number_source_type'].isnull()].shape[0] == len(product_number_dict)

In [55]:
devices_pd[['name_en', 'desc_en', 'model', 'product_number', 'product_additional', 'product_number_source_type']].rename({'model': 'model_id'}, axis=1).to_csv(FILE_PATH_MANUAL_PRODUCT_NUMBERS_RAW)

# Save enriched devices data

In [56]:
devices_pd.to_csv(FILE_PATH_PROCESSED_DEVICES)

# Download icons

In [ ]:
for index, row in tqdm.notebook.tqdm(devices_pd.iterrows(), total=devices_pd.shape[0]):
    icon_url = row['icon_real']
    if not icon_url:
        continue
    
    # Magic string for extracting extension from url instead of slow MIME Headers
    # But extensions is wrong lol
    img_ext = os.path.splitext(urllib.parse.urlparse(icon_url).path)[1]
    img_path = f'{os.path.join(ASSETS_ICONS_RAW_PATH, row["model"])}{img_ext}'
    
#     # Show real extensions
#     if os.path.isfile(img_path):
#         print(imghdr.what(img_path))
    
    if (not os.path.isfile(img_path) or not imghdr.what(img_path)) and row['icon_real']:
        try:
            s = f'wget --tries=10 -O {img_path} "{row["icon_real"]}"'
            output = subprocess.check_output(s, shell=True)
        except Exception as e:
            print(e)

  0%|          | 0/4241 [00:00<?, ?it/s]

--2022-01-13 00:36:12--  https://cnbj1.fds.api.xiaomi.com/iotweb-product-center/30358d6ac901f455aa4233db744d14f1_1621912102223.png?GalaxyAccessKeyId=AKVGLQWBOVIRQ3XLEW&Expires=9223372036854775807&Signature=REfDBCEHcEuctpF2PUYKpyHBcj0=
Resolving cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)... 47.241.161.86
Connecting to cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)|47.241.161.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15452 (15K) [text/plain]
Saving to: ‘../assets/icons/raw/ows.tow_w.mjj20a.png’

     0K .......... .....                                      100% 38,9M=0s

2022-01-13 00:36:15 (38,9 MB/s) - ‘../assets/icons/raw/ows.tow_w.mjj20a.png’ saved [15452/15452]

--2022-01-13 00:36:15--  https://cnbj1.fds.api.xiaomi.com/iotweb-product-center/f91c34bcb5120d6596e310de47635cb6_168X168-%E7%99%BD.png?GalaxyAccessKeyId=AKVGLQWBOVIRQ3XLEW&Expires=9223372036854775807&Signature=h3bRHJSU9ACqOetexpmMgA0jvEE=
Resolving cnbj1.fds.api.xiaomi.com (cn

Connecting to cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)|47.241.161.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14661 (14K) [text/plain]
Saving to: ‘../assets/icons/raw/babai.switch.202ml.png’

     0K .......... ....                                       100% 41,6M=0s

2022-01-13 00:36:37 (41,6 MB/s) - ‘../assets/icons/raw/babai.switch.202ml.png’ saved [14661/14661]

--2022-01-13 00:36:37--  https://cnbj1.fds.api.xiaomi.com/iotweb-product-center/8118469f1e9d60d3a5283b0ababdd05c_dengdai.png?GalaxyAccessKeyId=AKVGLQWBOVIRQ3XLEW&Expires=9223372036854775807&Signature=OzF1JgwbsR3rVlkYRSstipH3MQY=
Resolving cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)... 47.241.161.86
Connecting to cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)|47.241.161.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25926 (25K) [text/plain]
Saving to: ‘../assets/icons/raw/worth.light.wsomqn.png’

     0K .......... .......... .....        

Connecting to cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)|47.241.161.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13926 (14K) [text/plain]
Saving to: ‘../assets/icons/raw/babai.switch.203s.png’

     0K .......... ...                                        100% 9,29M=0,001s

2022-01-13 00:37:05 (9,29 MB/s) - ‘../assets/icons/raw/babai.switch.203s.png’ saved [13926/13926]

--2022-01-13 00:37:05--  https://cnbj1.fds.api.xiaomi.com/iotweb-product-center/7ab2417806d606eaebfb543c0858c64d_1624675019079.png?GalaxyAccessKeyId=AKVGLQWBOVIRQ3XLEW&Expires=9223372036854775807&Signature=ODXZuzJc/f8av/OlE+qD39pGVeY=
Resolving cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)... 47.241.161.86
Connecting to cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)|47.241.161.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12971 (13K) [text/plain]
Saving to: ‘../assets/icons/raw/topwit.switch.rzw02.png’

     0K .......... ..             

Connecting to cdn.cnbj1.fds.api.mi-img.com (cdn.cnbj1.fds.api.mi-img.com)|58.243.200.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17292 (17K) [text/plain]
Saving to: ‘../assets/icons/raw/bull.switch.s112y.png’

     0K .......... ......                                     100% 41,0K=0,4s

2022-01-13 00:37:29 (41,0 KB/s) - ‘../assets/icons/raw/bull.switch.s112y.png’ saved [17292/17292]

--2022-01-13 00:37:29--  https://cnbj1.fds.api.xiaomi.com/iotweb-product-center/40465ad8bf617e1c36ceefc673cb4ffe_%E5%8D%9A%E6%B5%81%E5%86%B7%E6%9A%96%E5%8F%8C%E8%89%B2%E7%81%AF.png?GalaxyAccessKeyId=AKVGLQWBOVIRQ3XLEW&Expires=9223372036854775807&Signature=bOLave+D2PJVZmPG+hLXPQKuJqE=
Resolving cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)... 47.241.161.86
Connecting to cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)|47.241.161.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10602 (10K) [text/plain]
Saving to: ‘../assets/icons/raw/coo

...

HTTP request sent, awaiting response... 200 OK
Length: 8415 (8,2K) [text/plain]
Saving to: ‘../assets/icons/raw/ldws.light.04001e.png’

     0K ........                                              100% 20,3M=0s

2022-01-13 00:58:40 (20,3 MB/s) - ‘../assets/icons/raw/ldws.light.04001e.png’ saved [8415/8415]

--2022-01-13 00:58:40--  https://cnbj1.fds.api.xiaomi.com/iotweb-product-center/b75b2b960e00b7604320c5cfbed8de5f_168.png?GalaxyAccessKeyId=AKVGLQWBOVIRQ3XLEW&Expires=9223372036854775807&Signature=kr7cifd7QWAxBHzdWo4OM/dNNiY=
Resolving cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)... 47.241.161.86
Connecting to cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)|47.241.161.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9306 (9,1K) [text/plain]
Saving to: ‘../assets/icons/raw/panpan.airer.y18.png’

     0K .........                                             100% 28,2M=0s

2022-01-13 00:58:42 (28,2 MB/s) - ‘../assets/icons/raw/panpan.airer.y18.png

Connecting to cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)|47.241.161.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34319 (34K) [text/plain]
Saving to: ‘../assets/icons/raw/bull.switch.s051a.png’

     0K .......... .......... .......... ...                  100% 36,3M=0,001s

2022-01-13 00:59:05 (36,3 MB/s) - ‘../assets/icons/raw/bull.switch.s051a.png’ saved [34319/34319]

--2022-01-13 00:59:05--  https://cnbj1.fds.api.xiaomi.com/iotweb-product-center/9d915b95cb20d8edc0f9c2a9b752031d_1621055305489.png?GalaxyAccessKeyId=AKVGLQWBOVIRQ3XLEW&Expires=9223372036854775807&Signature=8G/WKld9un4mfktszDozenQLBjw=
Resolving cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)... 47.241.161.86
Connecting to cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)|47.241.161.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29425 (29K) [text/plain]
Saving to: ‘../assets/icons/raw/090615.plug.cmp86.png’

     0K .......... .......... ......

HTTP request sent, awaiting response... 200 OK
Length: 4406 (4,3K) [text/plain]
Saving to: ‘../assets/icons/raw/viomi.waterheater.u51.png’

     0K ....                                                  100% 2,05G=0s

2022-01-13 00:59:50 (2,05 GB/s) - ‘../assets/icons/raw/viomi.waterheater.u51.png’ saved [4406/4406]

--2022-01-13 00:59:50--  https://cnbj1.fds.api.xiaomi.com/iotweb-product-center/c862a63f9de3bf7423193f173ee94193_1632193535754.png?GalaxyAccessKeyId=AKVGLQWBOVIRQ3XLEW&Expires=9223372036854775807&Signature=6iaAHua97QN0Bl4nbBnhTuj+w84=
Resolving cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)... 47.241.161.86
Connecting to cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)|47.241.161.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14140 (14K) [text/plain]
Saving to: ‘../assets/icons/raw/srkj.fan.fs01.png’

     0K .......... ...                                        100% 69,9M=0s

2022-01-13 00:59:53 (69,9 MB/s) - ‘../assets/icons/raw/srkj.

Connecting to cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)|47.241.161.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18914 (18K) [text/plain]
Saving to: ‘../assets/icons/raw/xiaomi.router.ra70.png’

     0K .......... ........                                   100% 42,7M=0s

2022-01-13 01:00:16 (42,7 MB/s) - ‘../assets/icons/raw/xiaomi.router.ra70.png’ saved [18914/18914]

--2022-01-13 01:00:16--  https://cnbj1.fds.api.xiaomi.com/iotweb-product-center/3ea52ce2e3ddb0c8fa7be48b42f42910_1621845679556.png?GalaxyAccessKeyId=AKVGLQWBOVIRQ3XLEW&Expires=9223372036854775807&Signature=YLRlSQ97Uo49EiICDkFVqLp2PY0=
Resolving cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)... 47.241.161.86
Connecting to cnbj1.fds.api.xiaomi.com (cnbj1.fds.api.xiaomi.com)|47.241.161.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5095 (5,0K) [text/plain]
Saving to: ‘../assets/icons/raw/zhiqin.switch.mij05.png’

     0K ....                        

Connecting to cdn.cnbj1.fds.api.mi-img.com (cdn.cnbj1.fds.api.mi-img.com)|58.243.200.177|:443... connected.


In [ ]:
# for img_name in os.listdir(ASSETS_ICONS_RAW_PATH):
#     try:
#         dest_img_path = os.path.join(ASSETS_ICONS_PROCESSED_PATH, img_name)
#         src_img_path = os.path.join(ASSETS_ICONS_RAW_PATH, img_name)
#         if (not os.path.isfile(dest_img_path) or not imghdr.what(dest_img_path)):
#             im = Image.open(src_img_path)
#             im.thumbnail([128, 128], Image.ANTIALIAS)
#             im.save(os.path.join(dest_img_path), "PNG")
#     except IOError:
#         print("cannot create thumbnail for '%s'" % item)

# Create Table

In [58]:
df = (
    devices_pd[['icon_real', 'name_en', 'desc_en', 'metacategory_name', 'metasubcategory_name', 'model', 'product_number']]
        .sort_values(['metacategory_name', 'metasubcategory_name', 'name_en', 'model'])
        .reset_index()
        .groupby(['metacategory_name', 'metasubcategory_name'])
        .apply(lambda x: pd.concat([x['icon_real'], x['name_en'], x['desc_en'], x['model'], x['product_number']], axis=1))
        .rename({
            'icon_real': 'Pic', 
            'name_en': 'Name', 
            'desc_en': 'Description', 
            'model': 'Model',
            'product_number': 'Product Number'
        }, axis=1)
        .fillna('')
#         .rename({'metacategory_name': 'Category', 'metasubcategory_name': 'Subcategory'}, axis=0)
)

# df['Pic'] = df['Pic'].apply(lambda x: f'<img src="{x}">')

img_pre = '../assets/icons/raw/'
# img_pre = '/home/home/Projects/mizoo/assets/icons/raw/'
df['Pic'] = df.apply(
    lambda x: f'<img src="{os.path.join(img_pre, x["Model"] + ".png")}">', 
    axis=1
)

# df.head()

In [59]:
with open('../markdown/devices.md', 'w') as f:
    print('# Mi-zoo\n', file=f)
    print('<sup><sub>Complete list of Xiaomi Mi Smart Home Ecosystem devices with descriptions and codes.</sub></sup>\n---', file=f)
    for category, df_sub in df.groupby(level=0):
        print(f'---\n# {category}', file=f)
        for subcategory, df_dev in df_sub.groupby(level=1):
            print(f'## {subcategory}', file=f)
            print(
                df_dev[['Pic', 'Name', 'Description', 'Model', 'Product Number']]
                .reset_index(level=['metasubcategory_name', 'metacategory_name'], drop=True)
                .to_html(escape=False)
                .replace('\\n', '<br />')
#                 .replace('<th>Pic</th>', '<th style="width:30%">Pic</th>')
#                 .replace('<th>Name</th>', '<th style="width:20%">Name</th>')
#                 .replace('<th>Model</th>', '<th style="width:10%">Model</th>')
#                 .replace('<th>Description</th>', '<th style="width:40%">Description</th>')
                ,file=f
            )
            print('', file=f)